In [2]:
import numba
import numpy as np
import os
from scipy.sparse import csr_matrix
import time

In [3]:
QueryFolderPath = 'ntust-ir-2020_hw4_v2/queries'
QueryNames = [pathimfor[2] for pathimfor in os.walk(QueryFolderPath)][0]
QueryPaths = [os.path.join(QueryFolderPath,QueryName) for QueryName in QueryNames]

In [4]:
DocumentFolderPath = 'ntust-ir-2020_hw4_v2/docs'
DocumentNames = [pathimfor[2] for pathimfor in os.walk(DocumentFolderPath)][0]
DocumentPaths = [os.path.join(DocumentFolderPath,DocumentName) for DocumentName in DocumentNames]

In [5]:
def GetValcabularyAndCalculateTF():
    Corpus_dict = {}
    Corpus_index = 0
    DocumentTF_list = []
    BackGround = {}
    
    T = 0
    
    for index,DocumentPath in enumerate(DocumentPaths):
        TF = {}
        with open(DocumentPath,'r') as file:
            Terms = file.read().split()
        
        for Term in Terms:
            TF[Term] = 1 if Term not in TF else TF[Term] + 1
            if Term not in Corpus_dict:
                Corpus_dict[Term] = Corpus_index
                Corpus_index += 1
            if Term not in BackGround:
                BackGround[Term] = 1
            else:
                BackGround[Term] += 1
        DocumentTF_list.append(TF)
    
        
        print(f"finish document:{index}",end = '\r')
        T += 1
        if T == 5000:
            break
    return Corpus_dict,DocumentTF_list,BackGround

In [6]:
# Conver TF Dictionary to List
def ConvertTFDict_to_List(Corpus_dict,DocumentTF_list):
    length = len(Corpus_dict)
    for index,DocumentTF in enumerate(DocumentTF_list):
        tempTF = [0]*length
        for Term in DocumentTF:
            tempTF[Corpus_dict[Term]] = DocumentTF[Term]
        DocumentTF_list[index] = tempTF
            
    return DocumentTF_list

In [7]:
def ConverBGTerm_to_ID(Corpus,BackGround):
    length = len(Corpus)
    tempBG = {}
    for Term in BackGround:
        tempBG[Corpus[Term]] = BackGround[Term]
    for Term in tempBG:
        tempBG[Term] /= sum(tempBG.values())
    return tempBG

In [8]:
class PLSA:
    def __init__(self,TopicNum,DocumentNum,WordNum,DocumentTF_list,Corpus,BackGround):
        self.TopicNum = TopicNum
        self.DocumentNum = DocumentNum
        self.WordNum = WordNum
        self.DocumentTF_list = DocumentTF_list
        self.Corpus = Corpus
        self.BackGround = BackGround
        self.P_TkWiDj = []
        self.P_WiTk = []
        self.P_TkDj = []
        
    @numba.jit()
    def InitialP_TkWiDj(self):
        for j in range(self.DocumentNum):
            temp_dict = {}
            for i in range(self.WordNum):
                temp_dict[i] = [1/TopicNum] * TopicNum
            self.P_TkWiDj.append(temp_dict)
        
    @numba.jit()
    def InitialP_WiTk(self):
        for k in range(self.TopicNum):
            self.P_WiTk.append([1/self.WordNum]*WordNum)
                
    @numba.jit()
    def InitialP_TkDj(self):
        for j in range(self.DocumentNum):
            self.P_TkDj.append([1/self.TopicNum] * TopicNum)
                
    @numba.jit()
    def Initial(self):
        self.InitialP_TkWiDj()
        self.InitialP_WiTk()
        self.InitialP_TkDj()

    @numba.jit()
    def E_step(self):
        for j in range(self.DocumentNum):
            for i in range(self.WordNum):
                Denominator = 0
                for k in range(self.TopicNum):
                    self.P_TkWiDj[j][i][k] = self.P_WiTk[k][i] * self.P_TkDj[j][k]
                    Denominator += self.P_TkWiDj[j][i][k]
                for k in range(self.TopicNum):
                    self.P_TkWiDj[j][i][k] /= Denominator
                    
    @numba.jit()
    def M_step(self):
        for k in range(self.TopicNum):
            Denominator = 0
            for i in range(self.WordNum):
                for j in range(self.DocumentNum):
                    if self.DocumentTF_list[j][i] !=0:
                        self.P_WiTk[k][i] += self.DocumentTF_list[j][i] * self.P_TkWiDj[j][i][k]
                Denominator += self.P_WiTk[k][i]
            for i in range(self.WordNum):
                self.P_WiTk[k][i] /= Denominator

        for j in range(self.DocumentNum):
            for k in range(self.TopicNum):
                for i in range(self.WordNum):
                    if self.DocumentTF_list[j][i] != 0:
                        self.P_TkDj[j][k] += self.DocumentTF_list[j][i] * self.P_TkWiDj[j][i][k]
                self.P_TkDj[j][k] /= sum(self.DocumentTF_list[j])
                
    def ComputeSimilarity(self,QueryTF,alpha,beta):
        Score = {}
        for j in range(self.DocumentNum):
            similarity = 1
            DocumentName = DocumentNames[j][:-4]
            for QueryIndex,(Queryi,QueryTF_Num) in enumerate(QueryTF.items()):
                if(Queryi != 'None'):
                    Document_value = self.DocumentTF_list[j][Queryi] / sum(self.DocumentTF_list[j])

                    PLSA_value = 0
                    for k in range(self.TopicNum):
                        PLSA_value += self.P_WiTk[k][Queryi] * self.P_TkDj[j][k]

                    BackGround_value =  self.BackGround[Queryi]


                    similarity *= alpha * Document_value + beta * PLSA_value + (1-alpha-beta)*BackGround_value
            Score[DocumentName] = similarity
        Score = sorted(Score.items(), key=lambda x:x[1])
        Score.reverse()
        return Score[:1000]
            
    @numba.jit()
    def CalculateLoss(self):
        loss = 0
        for j in range(self.DocumentNum):
            for i in range(self.WordNum):
                temp_loss = 0
                if self.DocumentTF_list[j][i] > 0:
                    for k in range(self.TopicNum):
                        temp_loss += self.P_WiTk[k][i] * self.P_TkDj[j][k]
                    if temp_loss > 0:
#                         temp_loss /= self.DocumentNum
                        loss += np.log(temp_loss) * self.DocumentTF_list[j][i]
        return -loss
                
    
    def EM_Algorithm(self,epochs):
        for epoch in range(epochs):
            self.E_step()
            finishtime = time.strftime('%X')
            self.M_step()
            finishtime = time.strftime('%X')
            loss = self.CalculateLoss()
            finishtime = time.strftime('%X')
            

In [ ]:
#GetValcabulary
Corpus,DocumentTF_list,BackGround = GetValcabularyAndCalculateTF()
# set parameter
TopicNum = 4
DocumentNum = len(DocumentNames)
WordNum = len(Corpus)
epochs = 50


In [ ]:
DocumentNum = len(DocumentNames)

In [ ]:
DocumentTF_list = ConvertTFDict_to_List(Corpus,DocumentTF_list)

In [ ]:
BackGround = ConverBGTerm_to_ID(Corpus,BackGround)

In [ ]:
TopicNum = 2
epochs = 6
Model = PLSA(TopicNum,DocumentNum,WordNum,DocumentTF_list,Corpus,BackGround)
# del DocumentTF_list
# del BackGround

In [ ]:
Model.Initial()

In [ ]:
Model.EM_Algorithm(epochs)

In [ ]:
def ComputeQueryTF(path):
    with open(path,'r') as file:
        Terms = file.read().split()
    TF = {}
    for term in Terms:
        if term not in TF:
            TF[term] = 1
        else:
            TF[term] += 1
    return TF

In [ ]:
def ConvertQueryTermDict_to_ID(Corpus_dict,QueryTF):
    tempTF = {}
    for term in QueryTF:
        if term in Corpus:
            tempTF[Corpus_dict[term]] = QueryTF[term]
        else:
            tempTF['None'] = 0
    return tempTF

In [ ]:
File = open('001.txt','w')
File.write('Query,RetrievedDocuments')
for index,path in enumerate(QueryPaths):
    print(f"start:{index}")
    QueryFileName = QueryNames[index][:-4]
    File.writelines('\n' + QueryFileName + ',')
    
    QueryTF = ComputeQueryTF(path)
    QueryTF = ConvertQueryTermDict_to_ID(Corpus,QueryTF)
    print(f"start compute")
    Score = Model.ComputeSimilarity(QueryTF,.8,.1)
    for (DocumentName,score) in Score:
        File.write(DocumentName + ' ')
File.close()